In [ ]:
#
# LLM with QnA and Google Search
#
#
!pip install youtube-transcript-api # YouTube lib to get transcript
!pip install pytube                 # YouTube lib
!pip install openai                 # OpenAI GPT
!pip install transformers           # Used to know the token size
!pip install chromadb==0.3.2
!pip install langchain==0.0.103
!pip install python-dotenv

In [ ]:
from dotenv import dotenv_values
api_keys = dotenv_values('keys.txt')

import os
os.environ['OPENAI_API_KEY'] = api_keys['OPENAI_API_KEY']
os.environ['SERPER_API_KEY'] = api_keys['SERPER_API_KEY']

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI()

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
db = Chroma.from_documents(docs, embeddings, persist_directory='db')

In [ ]:
query = "When is the next Devoxx Belgium? Answer truthfully and when you don't know the answer say : 'I don't know'"

In [ ]:
search_results = db.similarity_search(query)
search_results
len(search_results)

In [ ]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm, chain_type="stuff")
chain.run(input_documents=search_results, question=query)

In [ ]:
#
# Let's use Google
#
from langchain.agents import load_tools
from langchain.agents import initialize_agent

tools = load_tools(["google-serper"], llm=llm)

In [ ]:
query = "When is the next Devoxx Belgium?"

In [ ]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
agent.run(query)

In [ ]:
query = "When is Devoxx Belgium in 2023?"
agent.run(query)